## Mở đầu

**Chủ đề:** Nén và phát hiện chuỗi bất thường bằng Trie và phân tích thống kê

**Sinh viên:** Trần Tấn Phát, Nguyễn Anh Quân

***Colab notebook này phục vụ cho việc demo chương trình.***

## Thiết lập

**[Cần chỉnh sửa lệnh git clone khi merge về master]**

In [ ]:
# Clone repo
!git clone --branch readme-colab https://github.com/quannguyen2412899/SCAD-TSA.git
%cd SCAD-TSA

In [ ]:
# Cài đặt Graphviz và python pandas
!apt-get install graphviz
%pip install graphviz pandas

## Biên dịch

In [ ]:
!mkdir -p bin
# Biên dịch module preprocess
!g++ -std=c++17 -I./include src/preprocess.cpp src/Preprocessor.cpp -o bin/preprocess
# Biên dịch module analyze
!g++ -std=c++17 -I./include src/analyze.cpp src/Analysis.cpp src/StatTrie.cpp -o bin/analyze
# Biên dịch module visualize
!g++ -std=c++17 -I./include src/visualize.cpp -o bin/visualize
# Biên dịch chương trình chính
!g++ -std=c++17 -I./include src/main_pipeline.cpp -o main_pipeline

## Chuẩn bị dữ liệu


In [ ]:
!mkdir data results

In [ ]:
import random
import string

# Cấu hình file output
output_file = "data/sample_generated.log"

print("🔄 Đang sinh dữ liệu giả lập...")

with open(output_file, "w") as f:
    # ---------------------------------------------------------
    # 1. DỮ LIỆU BÌNH THƯỜNG (NORMAL TRAFFIC)
    # - Tần suất: Cao
    # - Độ dài: Trung bình, phổ biến
    # - Entropy: Thấp
    # ---------------------------------------------------------
    normal_patterns = [
        "INFO: GET /index.html 200",
        "INFO: GET /home.php 200",
        "INFO: GET /img/logo.png 200",
        "INFO: GET /img/banner.jpg 200",
        "INFO: POST /login 200"
    ]
    
    # Sinh 5000 dòng bình thường
    for _ in range(5000):
        f.write(random.choice(normal_patterns) + "\n")

    # ---------------------------------------------------------
    # 2. BẤT THƯỜNG TẦN SUẤT (FREQUENCY ANOMALY)
    # - Đặc điểm: Chuỗi duy nhất, chưa từng thấy.
    # ---------------------------------------------------------
    # Kẻ tấn công thử lệnh shutdown hệ thống (chỉ 1 lần)
    f.write("CRITICAL: SYSTEM SHUTDOWN INITIATED BY REMOTE HOST\n") 
    print("-> Gài 1 Bất thường Tần suất (Shutdown Command).")

    # ---------------------------------------------------------
    # 3. BẤT THƯỜNG ĐỘ DÀI (LENGTH ANOMALY - Rare Length)
    # - Đặc điểm: Độ dài mà 5000 dòng kia không có.
    # ---------------------------------------------------------
    # Tấn công Buffer Overflow: Chuỗi dài 500 ký tự 'A'
    buffer_overflow = "ERROR: Input buffer overflow " + ("A" * 500)
    f.write(buffer_overflow + "\n")
    print("-> Gài 1 Bất thường Độ dài (Buffer Overflow - 500 chars).")

    # ---------------------------------------------------------
    # 4. BẤT THƯỜNG ENTROPY CAO (ENTROPY ANOMALY)
    # - Đặc điểm: Tại một node, nhánh rẽ ra quá nhiều hướng (Hỗn loạn).
    # ---------------------------------------------------------
    # Tấn công Fuzzing: Thử 100 ký tự rác khác nhau vào tham số 'token='
    # Tiền tố chung: "WARN: Invalid token="
    # Hậu tố: Thay đổi liên tục -> Node sau dấu '=' sẽ có 100 nhánh con, mỗi nhánh count=1
    # => Entropy tại node '=' sẽ RẤT CAO.
    
    fuzzing_chars = string.ascii_letters + string.digits + "!@#$%^&*()"
    for i in range(100):
        # Chọn ngẫu nhiên 1 ký tự để tạo nhánh ngay lập tức
        char = fuzzing_chars[i % len(fuzzing_chars)] 
        f.write(f"WARN: Invalid token={char}...\n")
        
    print("-> Gài 100 dòng Bất thường Entropy Cao (Fuzzing Attack).")

print(f"✅ Đã tạo file: {output_file}")

## Chạy thử chương trình

#### Chạy pipeline

In [ ]:
# Phân tích dữ liệu sample, hiển thị cây trie thu gọn, chọn phần vị tần suất, độ dài và entropy lần lượt là P1, P1, P99
!./main_pipeline data/sample_generated.log results --visual-partial --perc-freq=1 --perc-len=1 --perc-entropy=99

#### Hiển thị kết quả
##### Báo cáo tổng quan

In [ ]:
!cat results/overall_report.txt

##### Danh sách bất thường

In [ ]:
# Danh sách bất thường tần suất
import pandas as pd
df = pd.read_csv("results/frequency_anomalies.csv")
display(df)

In [ ]:
# Danh sách bất thường độ dài
import pandas as pd
df = pd.read_csv("results/length_anomalies.csv")
display(df)

In [ ]:
# Danh sách bất thường entropy
import pandas as pd
df = pd.read_csv("results/entropy_anomalies.csv")
display(df)

Ngoài ra còn có danh sách tất cả các chuỗi/tiền tố được thống kê trong ```results/all_entries.csv```

In [ ]:
import pandas as pd
df = pd.read_csv("results/entropy_anomalies.csv")
display(df)

##### Hiển thị cây Trie

In [ ]:
from IPython.display import Image, display
display(Image("results/partial_trie.png"))